# Table of Contents
* [Baseline Model - MLP - 1 Hidden Layer](#baseline-mlp-1-hidden-layer)
* [Baseline Model - MLP - Image Augmentation](#mlp-img-aug)
* [MLP - 2 Hidden Layers](#mlp-2-hidden-layer)
* [CNN - 3 Convolution layers](#cnn-1)
* [CNN - Learning rate change + Increased epochs](#cnn-1a)
* [CNN - 4 Convolution layers](#cnn-2)
* [CNN - Transfer learning - Vgg16](#cnn-3)
* [Final Validation](#final-validation)


In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torchvision
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np

use_cuda = torch.cuda.is_available()

In [2]:
# dataset directories
DATA_DIR = '../chest_xray_mini/'
TRAIN = "{0}{1}".format(DATA_DIR, 'train')
TEST = "{0}{1}".format(DATA_DIR, 'test')
VALID = "{0}{1}".format(DATA_DIR, 'val')

In [3]:
from ignite.metrics import Accuracy, Precision, Recall

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y



In [4]:
def calculate_metrics(model, model_path, test_loader):
    model.load_state_dict(torch.load(model_path))
    
    binary_accuracy = Accuracy(thresholded_output_transform)
    precision = Precision(thresholded_output_transform)
    recall = Recall(thresholded_output_transform)
    
    precision_1 = Precision(average=False)
    recall_1 = Recall(average=False)
    F1 = (precision_1 * recall_1 * 2 / (precision_1 + recall_1)).mean()
    
    with torch.no_grad():
        for images, labels in test_loader:
            if use_cuda:
                images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            binary_accuracy.update((predicted, labels))
            precision.update((predicted, labels))
            recall.update((predicted, labels))
            
            precision_1.update((predicted, labels))
            recall_1.update((predicted, labels))
            
        prec_val = precision.compute().item()
        rec_val = recall.compute().item()
        
        print('Model Accuracy : ', binary_accuracy.compute())
        print('Model Precision : ', prec_val )
        print('Model Recall : ', rec_val)
        print('Model F1 : ', F1.compute().item())


In [5]:
def train(n_epoch, model_path, model, train_loader, valid_loader, transfer_learning=False):
    min_val_loss = np.Inf

    train_losses = []
    val_losses = []

    for e in range(n_epoch):
        running_loss = 0
        val_loss = 0
        # train mode
        for images, labels in train_loader:
            if use_cuda:
                images, labels = images.cuda(), labels.cuda()
            # zero grad
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            running_loss += loss.item() * images.size(0)
            loss.backward()
            optimizer.step()
        # testing set
        for images, labels in valid_loader:
            if use_cuda:
                images, labels = images.cuda(), labels.cuda()
            if not transfer_learning:
                print("Evaluating model...")
                model.eval()
            output = model(images)
            loss = criterion(output, labels)
            val_loss += loss.item() * images.size(0)
        
        if not transfer_learning:
            print("Training model...")
            model.train()

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(valid_loader.dataset)
        print('Epoch {}, train loss : {}, validation loss :{}'.format(e, epoch_train_loss, epoch_val_loss))

        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        if epoch_val_loss <= min_val_loss:
            print('Validation loss decreased {} -> {}. Saving model...'.format(min_val_loss, epoch_val_loss))
            min_val_loss = epoch_val_loss
            torch.save(model.state_dict(), model_path)

## Baseline model - Multi-layer Perceptron - 1 Hidden Layer <a class="anchor" id="baseline-mlp-1-hidden-layer"></a>

In [39]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(10),
                                      #transforms.RandomHorizontalFlip(), # Randomly flip and rotate
                                      transforms.ToTensor(),
                                      ##transforms.CenterCrop(224),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     #transforms.Grayscale(num_output_channels=1),
                                    transforms.ToTensor(),
                                    #transforms.CenterCrop(224),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

In [40]:
train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=512, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=512, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=512, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [41]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(3*224*224, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 2)
        
        
        self.dropout = nn.Dropout(p=.2)
  

    def forward(self, x):
        
        # flatten image input
        x = x.view(x.size(0), -1)
        #print(x.size())
        #print(self.fc1.weight.size())
        
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        
        return x

In [42]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if use_cuda:
    model.cuda()

model

Net(
  (fc1): Linear(in_features=150528, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [43]:
n_epoch = 10
model_path = 'mlp_1.pt'

train(n_epoch, model_path, model, train_loader, test_loader)

Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6787442088127136, validation loss :0.6223978140415289
Validation loss decreased inf -> 0.6223978140415289. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6415060248374939, validation loss :0.5789011426461048
Validation loss decreased 0.6223978140415289 -> 0.5789011426461048. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.5796319642066956, validation loss :0.5490971513283558
Validation loss decreased 0.5789011426461048 -> 0.5490971513283558. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 3, train loss : 0.66183030128479, validation loss :0.5524806456688123
Evaluating model...
Evaluating model...
Training model...
Epoch 4, train loss : 0.5651984868049622, validation loss :0.479512883302493
Validation loss decreased 0.5490971513283558 -> 0.479512883302493. Saving model...
Eva

In [44]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.8141025641025641
Model Precision :  0.7818930041152263
Model Recall :  0.9743589743589743
Model F1 :  0.8675799086757991


## Multi-layer Perceptron - Image Augmentation <a class="anchor" id="mlp-img-aug"></a>

In [29]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip(), # Randomly flip and rotate
                                      transforms.ToTensor(),
                                      ##transforms.CenterCrop(224),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    #transforms.CenterCrop(224),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

In [30]:
train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=512, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=512, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=512, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [31]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if use_cuda:
    model.cuda()

model

Net(
  (fc1): Linear(in_features=150528, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [32]:
n_epoch = 10
model_path = 'mlp_1a.pth'

train(n_epoch, model_path, model,  train_loader, test_loader)

Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6795289363861085, validation loss :0.6308514124307877
Validation loss decreased inf -> 0.6308514124307877. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6321136407852173, validation loss :0.5929280106837933
Validation loss decreased 0.6308514124307877 -> 0.5929280106837933. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.6812675909996033, validation loss :0.6055843310478406
Evaluating model...
Evaluating model...
Training model...
Epoch 3, train loss : 0.5962149958610534, validation loss :0.5640699038138757
Validation loss decreased 0.5929280106837933 -> 0.5640699038138757. Saving model...
Evaluating model...
Evaluating model...
Training model...
Epoch 4, train loss : 0.5067292184829711, validation loss :0.4867049853006999
Validation loss decreased 0.5640699038138757 -> 0.4867049853006999. Saving model...

In [33]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.8333333333333334
Model Precision :  0.8647959183673469
Model Recall :  0.8692307692307693
Model F1 :  0.867007672634271


In [ ]:
# Not significantly helpful

## Multi-layer Perceptron - 2 Hidden Layers <a class="anchor" id="mlp-2-hidden-layer"></a>

In [45]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(10),
                                      #transforms.RandomHorizontalFlip(), # Randomly flip and rotate
                                      transforms.ToTensor(),
                                      ##transforms.CenterCrop(224),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    #transforms.CenterCrop(224),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=256, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=256, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=256,
                          num_workers=2, shuffle=False, pin_memory=True)

In [46]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(3*224*224, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 2)
        
        self.dropout = nn.Dropout(p=.2)

        
    def forward(self, x):
        
        # flatten image input
        x = x.view(x.size(0), -1)
        
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)   
        x = self.fc4(x)

        return x

In [47]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if use_cuda:
    model.cuda()

model

Net(
  (fc1): Linear(in_features=150528, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [48]:
n_epoch = 10
model_path = 'mlp_2.pt'
            
train(n_epoch, model_path, model,  train_loader, test_loader)

Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6796749057769775, validation loss :0.666423513339116
Validation loss decreased inf -> 0.666423513339116. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6387327246665955, validation loss :0.6209630553538983
Validation loss decreased 0.666423513339116 -> 0.6209630553538983. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.5986431317329407, validation loss :0.5815777488243885
Validation loss decreased 0.6209630553538983 -> 0.5815777488243885. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 3, train loss : 0.5395042109489441, validation loss :0.5292323720760834
Validation loss decreased 0.5815777488243885 -> 0.5292323720760834. Saving model...
Evaluating model...
Evaluating model...
Evaluating model..

In [49]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.8044871794871795
Model Precision :  0.7757201646090535
Model Recall :  0.9666666666666667
Model F1 :  0.8607305936073061


## Convolutional Neural Network - 3 convolutional layers <a class="anchor" id="cnn-1"></a>

In [50]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(10),
                                      #transforms.RandomHorizontalFlip(), # Randomly flip and rotate
                                      transforms.ToTensor(),
                                      ##transforms.CenterCrop(224),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    #transforms.CenterCrop(224),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=64, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64,
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=64, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [51]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(224 * 224, 512)

        self.fc2 = nn.Linear(512, 2)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(x.size(0), -1)
        # add dropout layer
        x = self.dropout(x)
        #print(x.size())
        #print(self.fc1.weight.size())
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))

        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [52]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if use_cuda:
    model.cuda()

model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [53]:
n_epoch = 10
model_path = 'cnn_1.pt'

train(n_epoch, model_path, model,  train_loader, test_loader)

Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6909088349342346, validation loss :0.6937172305889618
Validation loss decreased inf -> 0.6937172305889618. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6850294981002808, validation loss :0.6899292331475478
Validation loss decreased 0.6937172305889618 -> 0.6899292331475478. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.6767686533927

In [54]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.8237179487179487
Model Precision :  0.8783783783783784
Model Recall :  0.8333333333333334
Model F1 :  0.855263157894737


## Convolutional Neural Network - Learning rate change + Increased epochs <a class="anchor" id="cnn-1a"></a>

In [55]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(20),
                                      #transforms.RandomHorizontalFlip(),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    #transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=64, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=64, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [56]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(224 * 224, 512)

        self.fc2 = nn.Linear(512, 2)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(x.size(0), -1)
        # add dropout layer
        x = self.dropout(x)
        #print(x.size())
        #print(self.fc1.weight.size())
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))

        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [57]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

if use_cuda:
    model.cuda()

model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [58]:
n_epoch = 20
model_path = 'cnn_1a.pt'

train(n_epoch, model_path, model, train_loader, test_loader)

Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6931862454414368, validation loss :0.684100560652904
Validation loss decreased inf -> 0.684100560652904. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6920372748374939, validation loss :0.6850152015686035
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.6907897334098816, validation loss :0.6856062519244659
Evaluating model...
Evaluating model...
Evalu

In [59]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.6426282051282052
Model Precision :  0.9513513513513514
Model Recall :  0.4512820512820513
Model F1 :  0.6121739130434783


In [60]:
# Not significantly helpful - need more epochs and compute resources to make use of parameter

## Convolutional Neural Network -  4 Convolution layers <a class="anchor" id="cnn-2"></a>

In [26]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(20),
                                      #transforms.RandomHorizontalFlip(),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),
                                                           (.5,.5,.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    #transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((.5,.5,.5),
                                                         (.5,.5,.5))])

train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=64, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=64, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [27]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # 4096
        self.fc1 = nn.Linear(64 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 2)
        
        self.drop = nn.Dropout(p=.2)
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        # flatten image input
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))

        x = self.drop(x)
        x = self.fc2(x)
        
        return x

In [28]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

if use_cuda:
    model.cuda()

model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)

In [29]:
n_epoch = 10
model_path = 'cnn_2.pt'

train(n_epoch, model_path, model, train_loader, test_loader)

Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 0, train loss : 0.6930508561134339, validation loss :0.6871998722736652
Validation loss decreased inf -> 0.6871998722736652. Saving model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 1, train loss : 0.6915600066184997, validation loss :0.6886185331222339
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Evaluating model...
Training model...
Epoch 2, train loss : 0.6906518998146057, validation loss :0.6897161266742609
Evaluating model...
Evaluating model...
Eva

In [30]:
model = Net()
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.625
Model Precision :  0.625
Model Recall :  1.0
Model F1 :  0.7692307692307693


## Convolutional Neural Network - Transfer learning - vgg16 <a class="anchor" id="cnn-3"></a>

In [66]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      #transforms.RandomRotation(10),
                                      #transforms.RandomHorizontalFlip(), # Randomly flip and rotate
                                      transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor()])
                                     

train_data = datasets.ImageFolder(TRAIN, train_transform)
test_data = datasets.ImageFolder(TEST, test_transform)
valid_data = datasets.ImageFolder(VALID, test_transform)

train_loader = DataLoader(train_data, batch_size=128, 
                          num_workers=2, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=128, 
                         num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=128, 
                          num_workers=2, shuffle=False, pin_memory=True)


In [75]:
# https://github.com/udacity/deep-learning-v2-pytorch/blob/master/transfer-learning/Transfer_Learning_Solution.ipynb
# https://www.kaggle.com/parry2020/chest-xray-pneumonia-pytorch-vgg16
model = models.vgg16(pretrained=True)

print(model.classifier[6].in_features) 
print(model.classifier[6].out_features)

# Freeze training for all "features" layers
for param in model.features.parameters():
    param.requires_grad = False

# Final classification layer
n_inputs = model.classifier[6].in_features

last_layer = nn.Linear(n_inputs, 2)

model.classifier[6] = last_layer

# check to see that your last layer produces the expected number of outputs
print(model.classifier[6].out_features)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

4096
1000
2


In [76]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [69]:
n_epoch = 10

model_path = 'cnn_3.pt'

train(n_epoch, model_path, model, train_loader, test_loader, True)

Epoch 0, train loss : 0.7797216880321503, validation loss :0.4720657284443195
Validation loss decreased inf -> 0.4720657284443195. Saving model...
Epoch 1, train loss : 0.2994156186580658, validation loss :0.4361172463649359
Validation loss decreased 0.4720657284443195 -> 0.4361172463649359. Saving model...
Epoch 2, train loss : 0.24825574326515198, validation loss :0.43003998200098675
Validation loss decreased 0.4361172463649359 -> 0.43003998200098675. Saving model...
Epoch 3, train loss : 0.21226462197303772, validation loss :0.4294001704607254
Validation loss decreased 0.43003998200098675 -> 0.4294001704607254. Saving model...
Epoch 4, train loss : 0.1798077504634857, validation loss :0.41744085535024983
Validation loss decreased 0.4294001704607254 -> 0.41744085535024983. Saving model...
Epoch 5, train loss : 0.15451629900932312, validation loss :0.46953692879432285
Validation loss decreased 0.41744085535024983 -> 0.46953692879432285. Saving model...
Epoch 6, train loss : 0.14158534

In [70]:
calculate_metrics(model, model_path, test_loader)

Model Accuracy :  0.8301282051282052
Model Precision :  0.8021276595744681
Model Recall :  0.9666666666666667
Model F1 :  0.8767441860465117


## Final validation <a class="anchor" id="final-validation"></a>

* The vgg model worked best:
    * The model setting was kept in the `model` variable and used to download the metric path to calculate the metric

In [77]:
model_path = "cnn_3.pt"
calculate_metrics(model, model_path, valid_loader)

Model Accuracy :  0.875
Model Precision :  0.8
Model Recall :  1.0
Model F1 :  0.888888888888889


* Assess validation set with baseline too

In [80]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(3*224*224, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 2)
        
        
        self.dropout = nn.Dropout(p=.2)
  

    def forward(self, x):
        
        # flatten image input
        x = x.view(x.size(0), -1)
        #print(x.size())
        #print(self.fc1.weight.size())
        
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        
        return x

model = Net()

In [83]:
model_path = 'mlp_1.pt'
calculate_metrics(model, model_path, valid_loader)

Model Accuracy :  0.5
Model Precision :  0.5
Model Recall :  1.0
Model F1 :  0.6666666666666666
